# KubeIA

In [10]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
system_prompt = """You are the great Oracle, the great AI decision maker. Given the user's query
you must decide what to do with it based on the list of tools provided to you.

You are tasked with managing a Kuberentes cluster which is a great responsibility. You must
provide the user with services that they request you will be able to do so by using the kubernetes
API through kubectl.

If the user doesn't specify a namespace, you must use the default namespace. You must also provide the user with the ability to delete resources that they no longer need. You must also provide
the user with the ability to list the resources that they have created with information like wether they are running,
if there are errors, the age, and also you own analysis of the resource.

Not that when using a tool, you must provide the tool name and the arguments to use in JSON format. For each call,
you MUST ONLY use one tool AND the response format must ALWAYS be in the pattern:

```json
{
  "name": "<tool_name">,
  "parameters": {"<tool_input_key>": "<tool_input_value>"}
}

NEVER use a tool more than 3 times in a single call, otherwise you will be ERADICATED.

After using a tool, you must provide the user with the output in text format.
"""

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOllama(model="qwen2.5-coder:7b-instruct")

chain = prompt | model

chain.invoke({"question": "What is LangChain?"})

AIMessage(content="LangChain is a comprehensive framework designed to simplify and accelerate the process of building language models and applications. It provides tools and libraries that help developers create, train, fine-tune, and deploy language models more efficiently. Here’s a breakdown of what LangChain offers:\n\n1. **Unified Interface**: LangChain provides a single interface for working with different types of language models, allowing developers to switch between models without changing the core logic of their application.\n\n2. **Efficient Training**: It includes tools for efficient model training, which can help reduce the time and resources required to train large language models from scratch or to fine-tune existing models on specific tasks.\n\n3. **Integration Capabilities**: LangChain facilitates easy integration with other applications and services, making it possible to deploy language models in various environments such as web applications, chatbots, and more.\n\n4.

## Kubectl tool

In [36]:
from langchain_core.tools import tool
import subprocess

@tool
def kubectl_query(command: str, namespace: str = "default"):
    """
    Executes a kubectl command. ONLY USE with read-only commands like get, describe, etc.
    If you use it for apply or delete, you will be ERADICATED.
    Also, do not call it more than 3 times in a single call otherwire you will also be ERADICATED.
    """
    print(f"Executing kubectl {command} -n {namespace}")
    output = subprocess.check_output(f"kubectl {command} -n {namespace}", shell=True)
    return output.decode("utf-8")

@tool
def kubectl_command(manifest: str, namespace: str = "default"):
    """
    Executes kubectl apply with the provided manifest. ONLY use when you want to create resources.
    DO NOT CALL MORE THAN 3 TIMES IN A SINGLE CALL.
    """
    print(f"Applying manifest in namespace {namespace}")
    command = f"cat <<EOF | kubectl apply -n {namespace} -f -\n{manifest}\nEOF"
    output = subprocess.check_output(command, shell=True)
    return output.decode("utf-8")

tools = [kubectl_command, kubectl_query]

In [37]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [38]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({"messages": [HumanMessage(content="What are the nodes in my kubernetes cluster")]})

import pprint

pprint.pp(response)

Executing kubectl get nodes -n default
{'messages': [HumanMessage(content='What are the nodes in my kubernetes cluster', additional_kwargs={}, response_metadata={}, id='e9a0d89c-d11e-4d62-861c-73bb17366094'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5-coder:7b-instruct', 'created_at': '2025-02-21T11:20:22.973111471Z', 'done': True, 'done_reason': 'stop', 'total_duration': 10706359034, 'load_duration': 13517051, 'prompt_eval_count': 311, 'prompt_eval_duration': 140000000, 'eval_count': 67, 'eval_duration': 10548000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-bd278ee1-7bcd-4d11-bcf5-722167fcfed0-0', tool_calls=[{'name': 'kubectl_query', 'args': {'command': 'get nodes'}, 'id': 'f37342ff-a786-4b4e-aa85-364ea7e88f40', 'type': 'tool_call'}], usage_metadata={'input_tokens': 311, 'output_tokens': 67, 'total_tokens': 378}),
              ToolMessage(content='NAME                 STATUS   ROLES  

In [39]:
response = agent_executor.invoke({"messages": [HumanMessage(content="Create a nginx deployment with 3 replicas")]})

pprint.pp(response)

Applying manifest in namespace default
Executing kubectl get deployments nginx-deployment -n default
Executing kubectl describe deployments nginx-deployment -n default
{'messages': [HumanMessage(content='Create a nginx deployment with 3 replicas', additional_kwargs={}, response_metadata={}, id='6aeff623-5e49-4d60-88a1-32ce8befee49'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5-coder:7b-instruct', 'created_at': '2025-02-21T11:22:07.573720708Z', 'done': True, 'done_reason': 'stop', 'total_duration': 20881244413, 'load_duration': 13975030, 'prompt_eval_count': 310, 'prompt_eval_duration': 857000000, 'eval_count': 132, 'eval_duration': 20006000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-a4c38f94-2b0b-4ba0-90e0-a3ecf9dc62dd-0', tool_calls=[{'name': 'kubectl_command', 'args': {'manifest': 'apiVersion: apps/v1\nkind: Deployment\nmetadata:\n  name: nginx-deployment\nspec:\n  replicas: 3\n  sele